In [ ]:
%pip install beacon-api --upgrade

In [ ]:
from beacon_api import *
import os
BLUE_CLOUD_JWT_TOKEN = os.getenv("BLUE_CLOUD_JWT_TOKEN") 
client = Client("https://beacon-cdi.maris.nl", jwt_token=BLUE_CLOUD_JWT_TOKEN)

In [ ]:
tables = client.list_tables()
tables

In [ ]:
tables['default'].get_table_schema()

In [ ]:
# METADATA
from datetime import datetime, timezone
dt_str = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

query_builder = tables['default'].query()
query_builder = (
    query_builder
    .add_select_column("__entry_key")
    .add_select(SelectLiteral("BEACON_SEADATANET", alias="SOURCE_BDI"))
    .add_select(SelectFunction("concat", [SelectColumn("SDN_EDMO_CODE"), SelectColumn("SDN_LOCAL_CDI_ID")], alias="SOURCE_BDI_DATASET_ID"))
    .add_select(SelectColumn("SDN_LOCAL_CDI_ID"))
    .add_select(SelectColumn("SDN_EDMO_CODE"))
    
    .add_select(SelectLiteral("2026-01-16T00:00:00Z", "COMMON_BDI_SNAPSHOT_DATE"))
    .add_select(SelectLiteral("1.5.0", "COMMON_BDI_MONOLITH_VERSION"))
    .add_select(SelectLiteral(dt_str, "COMMON_HARMONIZATION_DATE"))

    .add_select_column(".csr_platform_code")
    .add_select_column(".csr_platform_code", "COMMON_CSR")
    .add_select(SelectColumn(".instrument___gear_type"))
    .add_select(SelectColumn(".featureType"))
    .add_select(SelectColumn(".featureType", alias="COMMON_FEATURE_TYPE"))
    .add_select(SelectColumn(".originator", alias="COMMON_EDMO_CODE"))
    .add_select(SelectFunction("concat", [SelectColumn("SDN_EDMO_CODE"), SelectColumn("SDN_LOCAL_CDI_ID")], "COMMON_ODV_TAG"))
    .add_select(SelectColumn(".platform_type"))
    .add_select(SelectFunction("map_seadatanet_platform_l06", [SelectColumn(".platform_type")], alias="COMMON_PLATFORM_L06"))
    .add_select(SelectFunction("nvl2", [
        SelectColumn(".csr_platform_code"),
        SelectFunction("concat", [
            SelectLiteral("SDN:C17::"),
            SelectColumn(".csr_platform_code")
            ]),
        SelectLiteral("")
        ],
        alias="COMMON_PLATFORM_C17"))
    
)

In [ ]:
query_builder = (
    query_builder
    .add_select(SelectColumn("TIME"))
    .add_select(SelectColumn("TIME_SEADATANET_QC"))
    .add_select(SelectColumn("TIME", alias="COMMON_TIME"))
    .add_select(SelectFunction("cast_int8_as_char",[SelectColumn("TIME_SEADATANET_QC")], alias="COMMON_TIME_QC"))

    .add_select(SelectColumn("LATITUDE"))
    .add_select(SelectColumn("LATITUDE.standard_name"))
    .add_select(SelectColumn("LATITUDE.units"))
    .add_select(SelectColumn("LATITUDE", alias="COMMON_LATITUDE"))
    .add_select(SelectLiteral("degrees_north", "COMMON_LATITUDE_UNITS"))
    .add_select(SelectLiteral("Latitude", "COMMON_LATITUDE_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ALATZZ01", "COMMON_LATITUDE_P01"))
    .add_select(SelectLiteral("SDN:P06::DEGN", "COMMON_LATITUDE_P06"))
    
    .add_select(SelectColumn("LONGITUDE"))
    .add_select(SelectColumn("LONGITUDE.standard_name"))
    .add_select(SelectColumn("LONGITUDE.units"))
    .add_select(SelectColumn("LONGITUDE", alias="COMMON_LONGITUDE"))
    .add_select(SelectLiteral("degrees_east", "COMMON_LONGITUDE_UNITS"))
    .add_select(SelectLiteral("Longitude", "COMMON_LONGITUDE_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ALONZZ01", "COMMON_LONGITUDE_P01"))
    .add_select(SelectLiteral("SDN:P06::DEGE", "COMMON_LONGITUDE_P06"))
    
    .add_select(SelectColumn("POSITION_SEADATANET_QC"))
    .add_select(SelectFunction("cast_int8_as_char",[SelectColumn("POSITION_SEADATANET_QC")], alias="COMMON_POSITION_QC"))
    
    # DEPTH
    .add_select_column("PRES")
    .add_select_column("PRESPR01")
    .add_select_column("PRESPS02")
    .add_select_column("DEPTH")
    .add_select_column("ADEPZZ01")
    .add_select_column("DEPH")
    .add_select_column("DEPHPR01")
    
    .add_select_coalesced(["PRES", "PRESPR01", "PRESPS02", "DEPTH", "ADEPZZ01", "DEPH", "DEPHPR01"], alias="COMMON_ORIGIN_DEPTH")
    .add_select(SelectFunction("cast_int8_as_char", [
        Functions.coalesce(["PRES_SEADATANET_QC", 
                            "PRESPR01_SEADATANET_QC", 
                            "PRESPS02_SEADATANET_QC", 
                            "DEPTH_SEADATANET_QC", 
                            "ADEPZZ01_SEADATANET_QC", 
                            "DEPH_SEADATANET_QC", 
                            "DEPHPR01_SEADATANET_QC"], "merged_depth_qc")], 
                            alias="COMMON_ORIGIN_DEPTH_QC"))
    # COALESCED DEPTH
    .add_select(
        Functions.coalesce(
            [
                Functions.map_pressure_to_depth(
                    Functions.coalesce(["PRES",
                                        "PRESPR01",
                                        "PRESPS02"], alias = 'PRESSURE'),
                    "LATITUDE",
                    alias="depth_from_pres",
                ),  "DEPTH",
                    "ADEPZZ01",
                    "DEPH",
                    "DEPHPR01"
            ],
            alias="COMMON_DEPTH",
        )
    )
    .add_select(SelectFunction("cast_int8_as_char", [
        Functions.coalesce(["PRES_SEADATANET_QC", 
                            "PRESPR01_SEADATANET_QC", 
                            "PRESPS02_SEADATANET_QC", 
                            "DEPTH_SEADATANET_QC", 
                            "ADEPZZ01_SEADATANET_QC", 
                            "DEPH_SEADATANET_QC", 
                            "DEPHPR01_SEADATANET_QC"], "merged_depth_qc")], 
                            alias="COMMON_DEPTH_QC"))
    .add_select(SelectLiteral("m", "COMMON_DEPTH_UNITS"))
    .add_select(SelectLiteral("depth", "COMMON_DEPTH_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ADEPZZ01", "COMMON_DEPTH_P01"))
    .add_select(SelectLiteral("SDN:P06::ULAA", "COMMON_DEPTH_P06"))
)

In [ ]:
import numpy as np
# TEMPERATURE
query_builder = (
    query_builder
    .add_select(Functions.coalesce([
        Functions.try_cast_to_type(SelectColumn("PSSTTS01"), np.float64, "PSSTTS01_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPBT01"), np.float64, "TEMPBT01_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPCC01"), np.float64, "TEMPCC01_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPCC02"), np.float64, "TEMPCC02_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPCU01"), np.float64, "TEMPCU01_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPCU02"), np.float64, "TEMPCU02_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPDTNX"), np.float64, "TEMPDTNX_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPET01"), np.float64, "TEMPET01_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPHG01"), np.float64, "TEMPHG01_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPHG02"), np.float64, "TEMPHG02_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPHGNX"), np.float64, "TEMPHGNX_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPHU01"), np.float64, "TEMPHU01_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPMT01"), np.float64, "TEMPMT01_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPP901"), np.float64, "TEMPP901_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPPR01"), np.float64, "TEMPPR01_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPRTNX"), np.float64, "TEMPRTNX_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPS601"), np.float64, "TEMPS601_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPS901"), np.float64, "TEMPS901_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPST01"), np.float64, "TEMPST01_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPST02"), np.float64, "TEMPST02_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPSU01"), np.float64, "TEMPSU01_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPSZ01"), np.float64, "TEMPSZ01_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPTC01"), np.float64, "TEMPTC01_F64"),
        Functions.try_cast_to_type(SelectColumn("TEMPTC02"), np.float64, "TEMPTC02_F64"),
        Functions.try_cast_to_type(SelectColumn("TMCLCCI1"), np.float64, "TMCLCCI1_F64"),
        Functions.try_cast_to_type(SelectColumn("TMCLCCL1"), np.float64, "TMCLCCL1_F64")
    ], "COMMON_TEMPERATURE"))
    # .add_select_coalesced([
    #             "PSSTTS01",
    #             "TEMPBT01",
    #             "TEMPCC01",
    #             "TEMPCC02",
    #             "TEMPCU01",
    #             "TEMPCU02",
    #             "TEMPDTNX",
    #             "TEMPET01",
    #             "TEMPHG01",
    #             "TEMPHG02",
    #             "TEMPHGNX",
    #             "TEMPHU01",
    #             "TEMPMT01",
    #             "TEMPP901",
    #             "TEMPPR01",
    #             "TEMPRTNX",
    #             "TEMPS601",
    #             "TEMPS901",
    #             "TEMPST01",
    #             "TEMPST02",
    #             "TEMPSU01",
    #             "TEMPSZ01",
    #             "TEMPTC01",
    #             "TEMPTC02",
    #             "TMCLCCI1",
    #             "TMCLCCL1"], 
    #             alias="COMMON_TEMPERATURE")
    .add_select(SelectFunction("cast_int8_as_char", [
        Functions.coalesce(["PSSTTS01_SEADATANET_QC",
                "TEMPCC01_SEADATANET_QC",
                "TEMPCC02_SEADATANET_QC",
                "TEMPCU01_SEADATANET_QC",
                "TEMPCU02_SEADATANET_QC",
                "TEMPDTNX_SEADATANET_QC",
                "TEMPET01_SEADATANET_QC",
                "TEMPHG01_SEADATANET_QC",
                "TEMPHG02_SEADATANET_QC",
                "TEMPHGNX_SEADATANET_QC",
                "TEMPHU01_SEADATANET_QC",
                "TEMPMT01_SEADATANET_QC",
                "TEMPP901_SEADATANET_QC",
                "TEMPPR01_SEADATANET_QC",
                "TEMPRTNX_SEADATANET_QC",
                "TEMPS601_SEADATANET_QC",
                "TEMPS901_SEADATANET_QC",
                "TEMPST01_SEADATANET_QC",
                "TEMPST02_SEADATANET_QC",
                "TEMPSU01_SEADATANET_QC",
                "TEMPSZ01_SEADATANET_QC",
                "TEMPTC01_SEADATANET_QC",
                "TEMPTC02_SEADATANET_QC",
                "TMCLCCI1_SEADATANET_QC",
                "TMCLCCL1_SEADATANET_QC"], "merged_temperature_qc")],
                alias="COMMON_TEMPERATURE_QC"))
    .add_select(SelectLiteral("degree_C", "COMMON_TEMPERATURE_UNITS"))
    .add_select(SelectLiteral("sea_water_temperature", "COMMON_TEMPERATURE_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::TEMPPR01", "COMMON_TEMPERATURE_P01"))
    .add_select(SelectLiteral("SDN:P06::UPAA", "COMMON_TEMPERATURE_P06"))
    .add_select(SelectFunction("map_seadatanet_instrument_l05", [SelectColumn(".instrument___gear_type")], alias="COMMON_TEMPERATURE_L05"))
    .add_select_coalesced([
            "PSSTTS01.sdn_instrument_urn",
            # "TEMPBT01.sdn_instrument_urn", # Not available in L22
            # "TEMPCC01.sdn_instrument_urn", # Not available in L22
            # "TEMPCC02.sdn_instrument_urn", # Not available in L22
            "TEMPCU01.sdn_instrument_urn",
            # "TEMPCU02.sdn_instrument_urn", # Not available in L22
            # "TEMPDTNX.sdn_instrument_urn", # Not available in L22
            "TEMPET01.sdn_instrument_urn",
            "TEMPHG01.sdn_instrument_urn",
            # "TEMPHG02.sdn_instrument_urn", # Not available in L22
            # "TEMPHGNX.sdn_instrument_urn", # Not available in L22
            "TEMPHU01.sdn_instrument_urn",
            "TEMPMT01.sdn_instrument_urn",
            "TEMPP901.sdn_instrument_urn",
            "TEMPPR01.sdn_instrument_urn",
            "TEMPRTNX.sdn_instrument_urn",
            "TEMPS601.sdn_instrument_urn",
            "TEMPS901.sdn_instrument_urn",
            "TEMPST01.sdn_instrument_urn",
            "TEMPST02.sdn_instrument_urn",
            # "TEMPSU01.sdn_instrument_urn", # Not available in L22
            "TEMPSZ01.sdn_instrument_urn",
            "TEMPTC01.sdn_instrument_urn",
            "TEMPTC02.sdn_instrument_urn",
            # "TMCLCCI1.sdn_instrument_urn", # Not available in L22
            # "TMCLCCL1.sdn_instrument_urn" # Not available in L22
        ], 
        alias="COMMON_TEMPERATURE_L22"
    )
)

In [ ]:
# SALINITY
query_builder = (
    query_builder
    .add_select(Functions.coalesce([
        Functions.try_cast_to_type(SelectColumn("ASLTZZ01"), np.float64, "ASLTZZ01_F64"),
        Functions.try_cast_to_type(SelectColumn("ODSDM021"), np.float64, "ODSDM021_F64"),
        Functions.try_cast_to_type(SelectColumn("PSALBSTX"), np.float64, "PSALBSTX_F64"),
        Functions.try_cast_to_type(SelectColumn("PSALCC01"), np.float64, "PSALCC01_F64"),
        Functions.try_cast_to_type(SelectColumn("PSALCC02"), np.float64, "PSALCC02_F64"),
        Functions.try_cast_to_type(SelectColumn("PSALCU01"), np.float64, "PSALCU01_F64"),
        Functions.try_cast_to_type(SelectColumn("PSALCU02"), np.float64, "PSALCU02_F64"),
        Functions.try_cast_to_type(SelectColumn("PSALPR01"), np.float64, "PSALPR01_F64"),
        Functions.try_cast_to_type(SelectColumn("PSALPR02"), np.float64, "PSALPR02_F64"),
        Functions.try_cast_to_type(SelectColumn("PSALST01"), np.float64, "PSALST01_F64"),
        Functions.try_cast_to_type(SelectColumn("PSALSU01"), np.float64, "PSALSU01_F64"),
        Functions.try_cast_to_type(SelectColumn("PSALSZ01"), np.float64, "PSALSZ01_F64"),
        Functions.try_cast_to_type(SelectColumn("PSALTC01"), np.float64, "PSALTC01_F64"),
        Functions.try_cast_to_type(SelectColumn("PSALTC02"), np.float64, "PSALTC02_F64"),
        Functions.try_cast_to_type(SelectColumn("PSALZZXX"), np.float64, "PSALZZXX_F64"),
        Functions.try_cast_to_type(SelectColumn("PSCLCCI1"), np.float64, "PSCLCCI1_F64"),
        Functions.try_cast_to_type(SelectColumn("PSCLCCL1"), np.float64, "PSCLCCL1_F64"),
        Functions.try_cast_to_type(SelectColumn("PSLTZZ01"), np.float64, "PSLTZZ01_F64"),
        Functions.try_cast_to_type(SelectColumn("SSALAGT1"), np.float64, "SSALAGT1_F64"),
        Functions.try_cast_to_type(SelectColumn("SSALBSTX"), np.float64, "SSALBSTX_F64"),
        Functions.try_cast_to_type(SelectColumn("SSALPR01"), np.float64, "SSALPR01_F64"),
        Functions.try_cast_to_type(SelectColumn("SSALSG01"), np.float64, "SSALSG01_F64"),
        Functions.try_cast_to_type(SelectColumn("SSALST01"), np.float64, "SSALST01_F64"),
    ], "COMMON_SALINITY"))
    # .add_select_coalesced([
    #             "ASLTZZ01",
    #             "ODSDM021",
    #             "PSALBSTX",
    #             "PSALCC01",
    #             "PSALCC02",
    #             "PSALCU01",
    #             "PSALCU02",
    #             "PSALPR01",
    #             "PSALPR02",
    #             "PSALST01",
    #             "PSALSU01",
    #             "PSALSZ01",
    #             "PSALTC01",
    #             "PSALTC02",
    #             "PSALZZXX",
    #             "PSCLCCI1",
    #             "PSCLCCL1",
    #             "PSLTZZ01",
    #             "SSALAGT1",
    #             "SSALBSTX",
    #             "SSALPR01",
    #             "SSALSG01",
    #             "SSALST01",], 
    #             alias="COMMON_SALINITY")
    .add_select(SelectFunction("cast_int8_as_char", [Functions.coalesce([
                "ASLTZZ01_SEADATANET_QC",
                "ODSDM021_SEADATANET_QC",
                "PSALBSTX_SEADATANET_QC",
                "PSALCC01_SEADATANET_QC",
                "PSALCC02_SEADATANET_QC",
                "PSALCU01_SEADATANET_QC",
                "PSALCU02_SEADATANET_QC",
                "PSALPR01_SEADATANET_QC",
                "PSALPR02_SEADATANET_QC",
                "PSALST01_SEADATANET_QC",
                "PSALSU01_SEADATANET_QC",
                "PSALSZ01_SEADATANET_QC",
                "PSALTC01_SEADATANET_QC",
                "PSALTC02_SEADATANET_QC",
                "PSALZZXX_SEADATANET_QC",
                "PSCLCCI1_SEADATANET_QC",
                "PSCLCCL1_SEADATANET_QC",
                "PSLTZZ01_SEADATANET_QC",
                "SSALAGT1_SEADATANET_QC",
                "SSALBSTX_SEADATANET_QC",
                "SSALPR01_SEADATANET_QC",
                "SSALSG01_SEADATANET_QC",
                "SSALST01_SEADATANET_QC"], "merged_salinity_qc")], 
                alias="COMMON_SALINITY_QC"))
    .add_select(SelectLiteral("sea_water_practical_salinity", "COMMON_SALINITY_STANDARD_NAME"))
    .add_select(SelectLiteral("dimensionless", "COMMON_SALINITY_UNITS"))
    .add_select(SelectLiteral("SDN:P01::PSLTZZ01", "COMMON_SALINITY_P01"))
    .add_select(SelectLiteral("SDN:P06::UUUU", "COMMON_SALINITY_P06"))
    .add_select(SelectFunction("map_seadatanet_instrument_l05", [SelectColumn(".instrument___gear_type")], alias="COMMON_SALINITY_L05"))
    .add_select_coalesced([
        # "ASLTZZ01.sdn_instrument_urn", # Not available in L22
        "ODSDM021.sdn_instrument_urn",
        "PSALBSTX.sdn_instrument_urn",
        "PSALCC01.sdn_instrument_urn",
        # "PSALCC02.sdn_instrument_urn", # Not available in L22
        # "PSALCU01.sdn_instrument_urn", # Not available in L22
        # "PSALCU02.sdn_instrument_urn", # Not available in L22
        "PSALPR01.sdn_instrument_urn",
        # "PSALPR02.sdn_instrument_urn", # Not available in L22
        "PSALST01.sdn_instrument_urn",
        # "PSALSU01.sdn_instrument_urn", # Not available in L22
        "PSALSZ01.sdn_instrument_urn",
        "PSALTC01.sdn_instrument_urn",
        "PSALTC02.sdn_instrument_urn",
        "PSALZZXX.sdn_instrument_urn",
        # "PSCLCCI1.sdn_instrument_urn", # Not available in L22
        # "PSCLCCL1.sdn_instrument_urn", # Not available in L22
        "PSLTZZ01.sdn_instrument_urn",
        # "SSALAGT1.sdn_instrument_urn", # Not available in L22
        "SSALBSTX.sdn_instrument_urn",
        # "SSALPR01.sdn_instrument_urn", # Not available in L22
        "SSALSG01.sdn_instrument_urn",
        "SSALST01.sdn_instrument_urn"
    ], alias="COMMON_SALINITY_L22")
)

In [ ]:
import os 
import json

# Make output dir
output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)
# for each year between 1950 and 2025
for year in range(1950, 2026):
    print(f"Processing year {year}")
    query_builder.filters = []
    query_builder = query_builder.add_range_filter("COMMON_TIME", f"{year}-01-01T00:00:00Z", f"{year}-12-31T23:59:59Z")
    query_builder = (
        query_builder.add_filter(OrFilter([
            IsNotNullFilter("COMMON_TEMPERATURE"),
            IsNotNullFilter("COMMON_SALINITY"),
        ]))
    )
    
    output_file = os.path.join(output_dir, f"seadatanet_{year}.parquet")
    try:
        query_builder.to_parquet(output_file)
        print(f"Saved to {output_file}")
    except Exception as e:
        print(f"Failed to process year {year}: {e}")